In [1]:
%matplotlib inline
import pandas as pd
pd.options.display.max_rows = 100
pd.options.display.max_colwidth = 200

from hearthstone.enums import CardClass
from hearthstone import cardxml
db = cardxml.load()[0]

import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')

In [2]:
raw_df = pd.read_csv('chess_brawl.csv', header=None, engine='python')

In [3]:
raw_df['split'] = raw_df[4].str.strip().str.split('|')

In [4]:
import itertools

def merge_splits(group):
    return [s for s in list(itertools.chain.from_iterable([plays for plays in group.values if plays])) if s]

splits = raw_df['split'].groupby(level=0).apply(merge_splits)

In [97]:
error_rows = []
max_turns = 4

def process_split(s):
    plays_dict = {}

    for play in s[0]:
        try:
            card, turn, *_ = play.split(':')
            name = db[card].name
            if name not in ['Castle'] and ((card == 'GAME_005') or ('KAR_A10' in card.upper())):
                if turn in plays_dict:
                    plays_dict[turn].append(name)
                    plays_dict[turn] = sorted(plays_dict[turn])
                else:
                    plays_dict[turn] = [name]
        except:
            error_rows.append(s)
            continue
                
    # Merge into a list
    if plays_dict.keys():
        turn_count = min(max((int(k) for k in plays_dict.keys())), max_turns)
        playlist = [None] * turn_count;
        for turn in range(1, turn_count + 1):
            playlist[turn - 1] = plays_dict.get(str(turn), None)
        return pd.Series([ playlist ], index = ['playlist'])

results = pd.DataFrame(splits).apply(process_split, axis=1)

IndexError: ('list index out of range', 'occurred at index 0007316d-f3d0-4660-9a28-85ad905c1019')

In [6]:
wins = raw_df.xs('PLAYER1', level=1).ix[:,0]
wins.index = wins.index.droplevel(level=1)

In [7]:
results = results.join(wins)
results.rename(columns={0: 'white_result'}, inplace=True)

In [8]:
ct = pd.crosstab(results['playlist'].apply(str), results['white_result'])
popularity = ct.sum(axis=1)
left = pd.DataFrame(ct['WON'] / popularity)
left.rename(columns={0: 'win_rate'}, inplace=True)
table = left.join(pd.DataFrame(popularity))
table.columns = ['win_rate', 'popularity']

best = table.sort_values('popularity', ascending=False)

from tabulate import tabulate
print(tabulate(best, ['opening'] + list(best.columns.values), tablefmt="pipe"))

| opening                                                                                                                |   win_rate |   popularity |
|:-----------------------------------------------------------------------------------------------------------------------|-----------:|-------------:|
| [['White Pawn'], ['Black Pawn', 'Black Pawn', 'The Coin'], ['White Pawn', 'White Pawn'], ['Black Pawn', 'Black Pawn']] |  0.369297  |        21392 |
| [['White Pawn'], ['Black Pawn'], ['White Pawn', 'White Pawn'], ['Black Rook', 'The Coin']]                             |  0.293194  |        14458 |
| [['White Pawn'], ['Black Pawn'], ['White Pawn'], ['Black Pawn', 'Black Pawn']]                                         |  0.314946  |         9608 |
| [['White Pawn'], ['Black Pawn', 'Black Pawn', 'The Coin'], ['White Pawn', 'White Pawn'], ['Black Pawn']]               |  0.758409  |         7641 |
| [['White Pawn'], ['Black Pawn', 'Black Pawn', 'The Coin'], ['White Pawn'], ['Black Pawn']]  

In [452]:
import igraph as ig
from igraph import Graph
import math
import numpy as np

# Aggregate moves into graph
g = Graph(directed=True)
g.add_vertex(name='root')

root = g.vs.find(name='root')
root['won'] = 0
root['games'] = 0

for row in ct.iterrows():
    won = row[1]['WON']
    games = sum(row[1]) - row[1]['PLAYING']
    playlist = eval(row[0])
    
    root = g.vs.find(name='root')
    root['won'] += won
    root['games'] += games
    root['depth'] = 0
    root['move'] = 'START'
        
    if playlist:
        if playlist[0] == ['Black Pawn']: # drop bad row
            continue
        
        current_node = root
        path = []
        for depth, play in enumerate(playlist):
            path.append(play)

            name = str(path)
            if g.vs.select(name=name):
                n = g.vs.select(name=name)[0]
            else:
                g.add_vertex(name=name)
                g.add_edge(current_node['name'], name)
                n = g.vs.find(name=name)
                n['won'] = 0
                n['games'] = 0
                n['depth'] = depth + 1
                n['move'] = '\n'.join(play) if play else ('WHITE' if (depth % 2) == 0 else 'BLACK') + ' PASS'
                edge = g.es[g.incident(n, mode='IN')[0]]
                edge['won'] = 0
                edge['games'] = 0
                edge['move'] = '\n'.join(play) if play else 'PASS'

            n['won'] += won
            n['games'] += games
            edge = g.es[g.incident(n, mode='IN')[0]]
            edge['won'] += won
            edge['games'] += games
            
            current_node = n

g.vs['label'] = ['%s\nWW: %.1f%%\nG: %.1f%%' % (move, won / games * 100, games / root['games'] * 100)
                 for move, won, games in zip(g.vs['move'], g.vs['won'], g.vs['games'])]


# depth-based sizing
#shrink_factor = 0.15
#g.vs['size'] = [60 * math.exp(depth * -shrink_factor) for depth in g.vs['depth']]
#g.vs['label_size'] = [10 * math.exp(depth * -shrink_factor) for depth in g.vs['depth']]

# popularity-based sizing
p = 0.10
g.vs['size'] = np.power(g.vs['games'] / root['games'], p) * 60
g.vs['label_size'] = np.power(g.vs['games'] / root['games'], p) * 9
for v in g.vs: # Shrink long labels slightly
    if len(str.split(v['move'], '\n')) >= 3:
        v['label_size'] *= 0.85

g.es['width'] = np.power(g.es['games'] / root['games'], p) * 3.0
#g.es['arrow_width'] = np.multiply(g.es['width'], 0.5)
g.es['arrow_size'] = np.multiply(g.es['width'], 0.3)

# Give black and white different shapes
g.vs['shape'] = ['circle' if depth % 2 == 0 else 'rectangle' for depth in g.vs['depth']]

# delete small nodes
to_delete_ids = [v.index for v in g.vs if v['games'] < 100] #root['games'] * 0.005
g.delete_vertices(to_delete_ids)

# Limit depth
g = g.subgraph(g.neighborhood(vertices=root, order=10, mode='OUT'))


# Color nodes by win-rate
import matplotlib as mpl
import matplotlib.cm as cm

wr = np.divide(g.vs['won'], g.vs['games'])
norm = mpl.colors.Normalize(vmin=min(wr), vmax=max(wr))
cmap = cm.spring
m = cm.ScalarMappable(norm=norm, cmap=cmap)
g.vs['color'] = [m.to_rgba(rate)[:3] for rate in wr]

# Color edges by best-move
vids, *_ = g.bfs(root.index, mode='OUT')
for v in g.vs[vids]:
    edges = g.es[g.incident(v.index, mode='OUT')]
    if edges:
        wr = np.divide(edges['won'], edges['games'])
        if v['depth'] % 2 == 1:
            wr = 1 - wr
        edges[np.argmax(wr)]['color'] = 'light sea green'

root['games']

101864

In [453]:
layout = g.layout_reingold_tilford_circular(mode="out", root=[0])
ig.plot(g, layout=layout, bbox=(750, 650), margin=30,
       target='chess_openings.svg')
